## Process species names that produced errors in bacdive API search by using genus name only
E.g. Synechococcus sp.

In [1]:
import bacdive
import pandas as pd
import numpy as np

In [2]:
with open('up.txt', 'r') as file:
    creds = [line.rstrip() for line in file]

In [3]:
client = bacdive.BacdiveClient(creds[0], creds[1])

-- Authentication successful --


In [4]:
##read in error file and remove sp. and numbers
fileerrors = 'species_not_processed.txt'
with open(fileerrors, 'r') as file:
    errors = [line.rstrip() for line in file]
len(errors)

1265

In [25]:
genus_names = []
for item in errors:
    if 'subsp.' in item:
        index = item.index('subsp.')
        item = item[:index]
        item = item.strip()
        print(item)
        genus_names.append(item)
    elif 'sp.' in item:
        item = item.replace('sp.', '')
        item = item.strip()
        genus_names.append(item)
        
genus_names = list(set(genus_names))
len(genus_names)

Actinoplanes awajinensis
Actinoplanes deccanensis
Actinoplanes deccanensis
Bacillus safensis
Brochothrix thermosphacta
Burkholderia cepacia
Caldanaerobacter subterraneus
Chryseobacterium viscerum
Chryseobacterium viscerum
Elizabethkingia miricola
Elizabethkingia miricola
Enterobacter cloacae
Exiguobacterium acetylicum
Haemophilus intermedius
Haemophilus intermedius
Leuconostoc mesenteroides
Micromonospora mirobrigensis
Micromonospora sagamiensis
Micromonospora sagamiensis
Micromonospora sagamiensis
Nannocystis exedens
Neisseria elongata
Neisseria subflava
Nocardiopsis aegyptia
Pseudomonas nitroreducens
Rathayibacter tritici
Rhizobium rhizogenes
Rhodococcus obuensis
Sporosarcina ureae
Streptomyces ansochromogenes
Streptomyces aurantiogriseus
Streptomyces bikiniensis
Streptomyces biverticillatus
Streptomyces candidus
Streptomyces daghestianicus
Streptomyces flavotricini
Streptomyces griseus
Streptomyces griseus
Streptomyces griseus
Streptomyces humidus
Streptomyces hygroscopicus
Streptom

835

In [24]:
##test with genus name
filter = ['NCBI tax id', 'BacDive-ID', 'DSM-Number', 'species', 'Physiology and metabolism', 'Isolation, sampling and environmental information']
try:
    query = {'taxonomy': 'Streptomyces thermoolivaceus'}
    client.search(**query)
    result = client.retrieve(filter)
except Exception as e:
    print(e)
for item in result:
    print(item)

Your search did not receive any results.


KeyError: 'results'

In [21]:
##Search the filtered data for each unique species name in Bacdive and return NCBI ID, physiology & metabolism, and isolation data
filter = ['NCBI tax id', 'BacDive-ID', 'DSM-Number', 'species', 'Physiology and metabolism', 'Isolation, sampling and environmental information', 'Culture and growth conditions']
bacdive_data = {}
count = 0
error = []
for genus in genus_names:#[21:100]:
    bacdive_data[genus] = []
    try:
        query = {'taxonomy': genus}
        client.search(**query)
        result = client.retrieve(filter)
        for item in result:
            bacdive_data[genus].append(item)
        #time.sleep(3)
    except Exception as e:
        error.append((e, genus))
    count += 1
    if count%100 == 0:
        print(count)
len(bacdive_data)

Your search did not receive any results.
Your search did not receive any results.


79

In [ ]:
##save data dictionary with new results
import pickle
with open('bacdivedata_genus_20230512.pickle', 'wb') as filep:
    pickle.dump(bacdive_data, filep)

In [ ]:
with open('bacdivedata_genus_20230512_errors.txt', 'w') as f:
    f.write(errors)
    f.close()

In [ ]:
len(error)

In [22]:
error

[(KeyError('results'), 'Streptomyces thermoolivaceus '),
 (KeyError('results'), 'Treponema medium ')]

In [ ]:
import pickle
bacdata = pickle.load("bacdivedata_ID_isolation_metabolism_20230321.pickle")